### Scraping Health Board's Posts and Articles

In this project, we focus on extracting posts and articles from Health Board forums using web scraping techniques. Health Board is a platform with a wealth of user-generated content on various health topics, making it an excellent resource for applications such as sentiment analysis, trend monitoring, and topic modeling.

#### Objectives
- **Data Collection**: Retrieve posts and articles from specific health-related categories on Health Board based on topic criteria (e.g., mental health, fitness).
- **Data Processing**: Clean and preprocess the extracted data to prepare it for analysis.
- **Data Storage**: Store the scraped data in a structured format, such as CSV or a database, for further analysis.

#### Tools and Technologies
- **Python**: The primary programming language for web scraping.
- **Beautiful Soup**: A library for parsing HTML and extracting data.
- **Requests**: A library for making HTTP requests to access web pages.
- **Pandas**: A data manipulation library to handle and analyze the scraped data.

#### Getting Started
1. **Set Up the Environment**: Install the necessary libraries using pip.
2. **Define Scraping Logic**: Write functions to scrape data from specific health categories on Health Board.
3. **Run the Scraper**: Execute the scraping script and monitor the data collection process.
4. **Analyze the Data**: Use Pandas to analyze the collected posts and articles for insights.

#### Conclusion
This project serves as a practical introduction to web scraping and data analysis using Python, providing valuable experience in handling real-world data from an online health community.


<p style="color:#FE4406;text-align:center;font-size:30px"> Scraping Health board's  Posts And Articles </p>

In [34]:
!pip install bs4
!pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# importing packages
import requests
from bs4 import BeautifulSoup

### Scraping Health Boards

In [2]:
## importing libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

<p style="color:#FFC107;text-align:left;font-size:20px"> Searching for Health Board's health related topics  </p>

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# Set up the Selenium WebDriver
driver = webdriver.Chrome()  # Ensure you have the correct WebDriver
communities = []

# URL to scrape
url = 'https://www.healthboards.com/boards/hbcategory.php'
driver.get(url)

# Allow the page to load
time.sleep(5)

def scrape_current_page():
    # Get the page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all posts in the page (use the common class or structure to target posts)
    communities_elements = soup.find_all('b')

    new_content_found = False
    titlesList=soup.find_all("font")
    
    for community_element in titlesList:
        try:
            category=(community_element.contents[0])
            aElement=((community_element.parent.parent.parent))
            for element in aElement.next_siblings:
                if(str(element).startswith("<li>")):
                    topicLink=(element.a["href"])
                    topicName=(element.a.contents[0])
                    # Ensure the list exists
                    element={}
                    element["topicName"]=topicName
                    element["topicLink"]=topicLink
                    if(element not in communities):
                        communities.append(element)  
        except Exception as e:
            print(f"Error processing post: {e}")

    return new_content_found
# Scrape content from the current page
content_found = scrape_current_page()
# Close the driver when done
driver.quit()



In [43]:
len(communities)

157

In [44]:
# convert the topicLinks list to a dataset 
import pandas as pd
communities=pd.DataFrame(communities)
communities.head()

,topicName,topicLink
0,Addison's Disease,https://www.healthboards.com/boards/forumdispl...
1,Arthritis,https://www.healthboards.com/boards/forumdispl...
2,Autoimmune Disorders,https://www.healthboards.com/boards/forumdispl...
3,Chronic Fatigue,https://www.healthboards.com/boards/forumdispl...
4,Diabetes,https://www.healthboards.com/boards/forumdispl...


In [45]:
communities=communities.to_csv("../data/healthBoardsTopics.csv")

In [21]:
## Scraping topics from the topics list 
import pandas as pd 
topicsList=pd.read_csv("../data/healthBoardsTopics.csv")

In [ ]:
topicsNotDone=pd.read_csv("../data/healthBordsPosts.csv")[:8468]["topicTag"]
topics=[]

In [58]:
topicsNotDone=[element.replace(" ","") for element in list(set(list(topicsNotDone)))]

In [61]:
indexes=[]

In [62]:
for index in range(len(pd.read_csv("../data/healthBoardsTopics.csv"))):
    if(topicsList.iloc[index]["topicName"].replace(" ","") in  topicsNotDone):
        indexes.append(index)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

posts = []

def get_total_pages(soup):
    page_info = soup.find("td", class_="vbmenu_control", style="font-weight:normal")
    if page_info:
        match = re.search(r"Page \d+ of (\d+)", page_info.text)
        if match:
            return int(match.group(1))
    return 1

def scrape_current_page(url, topicTag):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "threadslist")))
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    tableElement = soup.find("table", id="threadslist")
    if tableElement:
        trElements = tableElement.find_all("tr")
        for i in range(5, len(trElements)):
            try:
                content_found = trElements[i]
                author = content_found.find("div", class_="smallfont").contents[0].strip().replace(" ", "")
                if author != "Administrator":
                    element = {
                        "commentsLink": content_found.find("a", id=re.compile(".*thread_title.*"))['href'],
                        "postTitle": content_found.find("a", id=re.compile(".*thread_title.*")).contents[0],
                        "authorId": author,
                        "postId": content_found.find("a", id=re.compile(".*thread_title.*"))["id"].replace("thread_title_", ""),
                        "commentsCount": content_found.find_all("td", class_="alt2")[1]["title"].split("Replies:")[1].split(",")[0].strip(),
                        "createdAt": content_found.find_all("div", class_="smallfont")[1].contents[0].strip().replace(" ", "") + " " + content_found.find("span", class_="time").contents[0].strip().replace(" ", ""),
                        "collectedAt": datetime.now(),
                        "topicTag": topicTag
                    }
                    print(element)
                    posts.append(element)

                    if len(posts) % 20 == 0:
                        return True
            except Exception as e:
                print(f"Error processing post: {e}")
                continue
    return False

for  index in indexes[1:]:
    topic_url = topicsList.loc[index, "topicLink"]
    topic_tag = topicsList.loc[index, "topicName"]
    
    driver.get(topic_url)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "threadslist")))
    first_page_soup = BeautifulSoup(driver.page_source, 'html.parser')
    total_pages = get_total_pages(first_page_soup)
    print("The number of total pages is ", total_pages)

    page_number = 2
    while page_number <= total_pages:
        if page_number == 1:
            current_page_url = topic_url
        else:
            current_page_url = f"{topic_url}index{page_number}.html"

        print(f"Navigating to: {current_page_url}")
        scrape_current_page(current_page_url, topic_tag)
        page_number += 1

    index += 1

driver.quit()


The number of total pages is  681
Navigating to: https://www.healthboards.com/boards/thyroid-disorders/index2.html
{'commentsLink': 'https://www.healthboards.com/boards/thyroid-disorders/106126-thyroid-symptom-list-other-information.html', 'postTitle': 'Thyroid Symptom List and Other Information', 'authorId': '', 'postId': '106126', 'commentsCount': '67', 'createdAt': '03-11-2007 07:55PM', 'collectedAt': datetime.datetime(2024, 11, 27, 9, 40, 40, 518557), 'topicTag': 'Thyroid Disorders'}
Error processing post: 'NoneType' object has no attribute 'contents'
{'commentsLink': 'https://www.healthboards.com/boards/thyroid-disorders/1053644-estrogen-change-thyroid.html', 'postTitle': 'Estrogen change and thyroid', 'authorId': 'Lady221', 'postId': '1053644', 'commentsCount': '0', 'createdAt': '12-04-2021 06:02PM', 'collectedAt': datetime.datetime(2024, 11, 27, 9, 40, 40, 520556), 'topicTag': 'Thyroid Disorders'}
{'commentsLink': 'https://www.healthboards.com/boards/thyroid-disorders/1053127-my

In [65]:
import pandas as pd 
posts=pd.DataFrame(posts)
previousCollected=pd.read_csv("../data/healthBordsPosts.csv")
data=pd.concat([posts,previousCollected])
data.to_csv("../data/healthBordsPosts.csv")